# CSE 5243 - Introduction to Data Mining
## Homework 5: Association Analysis
- Semester: AU23
- Instructor: Stanley Vernier
- Section: MON & WED, 11:10 AM
- Student Name: Chaeun Hong
- Student Email: hong.930@osu.edu
- Student ID: 500457492

Template Version V1.
***

# Introduction

### Objectives

In this lab, we will use the Instacart Market Basket Analysis dataset (see: https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis). However, we will use the data to solve a different (completely nonsense) problem.  The dataset CSV files are provided on Carmen.  They are:
- aisles.csv
- departments.csv
- products.csv
- order_products__train_**teb_subset**.csv  (This is a small'ish subset of the full Instacart transaction dataset. If you are brave, have free time on your hands and have a relatively powerful computer, you can try the full dataset, available at the URL listed above.  This is not necessary, though, and be sure to hand in the answers resulting from the smaller dataset.)

The objectives of this assignment are:
- Practice the Association Analysis content we covered this semester.
- Understand “why” the particular topics, techniques, etc., are important from a practical perspective.
- Understand how to choose and use appropriate tools to solve the provided problems.

### Dataset Notes
- The "order_products_..." data file captures the data in "long format". Specifically, every row corresponds to the transaction id and the item. If the specific transaction id has multiple items, there will be multiple rows in the data.
- You can process the data however you like, but it is recommended you convert into a one-hot-encoded data structure. This will allow you to easily use the mlxtend package.

## The Business Problem
- Assume the provided dataset contains all of the transactions for one month for our store (Trader Buck's).  We wish to find association rules that will improve our revenue as follows:
  - Each month, we will choose a particular **"Aisle of the Month"**. We will discount all products in that aisle by 10% each month, with the hope that this will encourage customers to visit our store to purchase those products 5% more frequently, and also purchase other products in other aisles (that are not discounted) more frequently.
  - For simplicity, assume the normal price of all products is $1.00 (a bargain!)
- Practically speaking, we want to come up with **two-item** rules (one antecedent and one consequent: (A -> B)) and choose the ones that best add to our revenues  (based on the rule support, confidence, etc.).

### Proper Answers
- **IMPORTANT:** **Show your work** and **explain it**.  This will help us give partial credit in some cases.

### Collaboration
For this assignment, you must work as an individual. You may informally discuss ideas with classmates, but your work must be your own.

### What You Need to Turn In
- Submit this Jupyter Notebook in .IPYNB format.  Do not "zip" the file.

### Notes
- Feel free to use the **mlxtend** package throughout this assignment.
- If a question asks you to "calculate" the number of "all possible rules", etc., explain the calculation by showing the "formula" you used. This will act as "showing your work".
***

***
# Section: 1 - Get Ready
1A) Load the data, and get it ready for association analysis. Do this with convenient python helper methods as appropriate. Feel free to use the tools given in the example we covered.
- Suggest: Make the data one-hot encoded.
***

In [2]:
#Note: If the mlxtend library is not installed, uncomment the following line (once) and run it.
!pip install mlxtend
import numpy as np
import pandas as pd
import mlxtend as mlx
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

In [3]:
# load the aisles data
aisles_df = pd.read_csv('aisles.csv')
display(aisles_df.info())
display(aisles_df.describe())
aisles_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


None

,aisle_id
count,134.000000
mean,67.500000
std,38.826537
min,1.000000
25%,34.250000
50%,67.500000
75%,100.750000
max,134.000000


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
# load the aisles data
departments_df = pd.read_csv('departments.csv')
display(departments_df.info())
display(departments_df.describe())
departments_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 464.0+ bytes


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


None

,department_id
count,21.000000
mean,11.000000
std,6.204837
min,1.000000
25%,6.000000
50%,11.000000
75%,16.000000
max,21.000000


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
# load the products data
products_df = pd.read_csv('products.csv')
display(products_df.info())
display(products_df.describe())
products_df.head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


None

,product_id,aisle_id,department_id
count,49688.000000,49688.000000,49688.000000
mean,24844.500000,67.769582,11.728687
std,14343.834425,38.316162,5.850410
min,1.000000,1.000000,1.000000
25%,12422.750000,35.000000,7.000000
50%,24844.500000,69.000000,13.000000
75%,37266.250000,100.000000,17.000000
max,49688.000000,134.000000,21.000000


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
# load the orders data (some columns)
orders_df = pd.read_csv('order_products__train_teb_subset.csv', usecols=['order_id','product_id'])
display(orders_df.info())
display(orders_df.describe())
orders_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163387 entries, 0 to 163386
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   order_id    163387 non-null  int64
 1   product_id  163387 non-null  int64
dtypes: int64(2)
memory usage: 2.5 MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


None

,order_id,product_id
count,163387.000000,163387.000000
mean,200333.473287,25545.946207
std,115314.521996,14125.982042
min,1.000000,1.000000
25%,99751.000000,13370.000000
50%,202141.000000,25146.000000
75%,299544.000000,37966.000000
max,399998.000000,49686.000000


,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [7]:
# merge two data (products_df and orders_df) to get the name of product (instead of product id)
merged_df = pd.merge(products_df, orders_df, on='product_id', how='inner')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# One-hot encoding of the orders data
from mlxtend.preprocessing import TransactionEncoder

te = mlx.preprocessing.TransactionEncoder()

a = merged_df.groupby('order_id')['product_name'].apply(list)
te_array = te.fit(a).transform(a)
ohe_df= pd.DataFrame(te_array, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
ohe_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,#2 Coffee Filters,#4 Natural Brown Coffee Filters,0 Calorie Fuji Apple Pear Water Beverage,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Strawberry Greek Yogurt,0% Fat Superfruits Greek Yogurt,0% Greek Strained Yogurt,...,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15466,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15467,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15468,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15469,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


***
# Section: 2 - Explore the Data
***

***
## Section: 2.1 - Get the Order and Product Sizes
- Calculate the **number_of_orders** and **number_of_products**.
***

In [ ]:
# calculate the number of orders an the number of products
number_of_orders = len(orders_df['order_id'].unique())
number_of_products = len(orders_df['product_id'].unique())

print("Number of orders: " + str(number_of_orders))
print("Number of products: " + str(number_of_products))

Number of orders: 15471
Number of products: 20605


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# calculate the number of products in the Instacart
number_of_products_whole = len(products_df['product_id'].unique())

print("Number of products in the Instcart: " + str(number_of_products_whole))

Number of products in the Instcart: 49688


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Discussion**

Based on the orders data, there are 15,741 orders and 20,605 different products. All products in the Instacart are 49,688.

***
## Section: 2.2 - Evaluate the Itemset and Rule Size & Complexity
- Calculate the **maximum number of Itemsets** that could be created from the items (without considering the actual transaction data). Show your work.
- Calculate the **maximum number of Rules** that can be created from the items (without considering the actual transaction data). Show your work.
- What do the calculations suggest as a **potential cause of concern**? Hint: Complexity.
- What might you do to manage these concerns?
***

**Discussion**

Calculate the maximum number of Itemsets that could be created from the items (without considering the actual transaction data).

> To calculate the maximum number of itemsets that could be created from the items, I need to calculate 2^(the number of products in the orders data). Therefore, the maximum number of itemsets is ***2^20605***, and it is out of limit using Python code because the number of products is too large.

Calculate the maximum number of Rules that can be created from the items (without considering the actual transaction data).

> To calculate the maximum number of rules that can be created from the items, I need to calculate 3^(the number of products in the orders data) - 2^(the number of products in the orders data + 1) + 1. Therefore, the maximum number of rules is ***3^20605 - 2^20605 + 1***, and it is also out of limit using Python code.

What do the calculations suggest as a potential cause of concern? What might you do to manage these concerns?

> The large number of items can lead to high computational complexity and time consumption when trying to find optimized rules. This is due to the many possible itemsets and rules that can be created. To manage these concerns, I can consider reducing the number of products (e.g., consider only frequently ordered products - using a threshold) or creating rules for aisles instead of products. This would reduce the maximum number of items and the maximum number of rules, thereby reducing computational complexity and time consumption. Additionally, while association rule, I need to consider ways to reduce the number of (itemset) candidates. For example, the apriori can be used to prune.


***
# Section: 3 - Itemset Generation
***

***
## Section: 3.1 - Revise the Dataset
- If/as appropriate, trim or revise the dataset to make the runtime reasonable.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [10]:
# merge two data (merged_df and aisles_df) to reduce the number of items
merged_df2 = pd.merge(merged_df, aisles_df, on='aisle_id', how='inner')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# one-hot encoding of the merged_df2 (aisle based)
a = merged_df2.groupby('order_id')['aisle'].apply(list)
te_array = te.fit(a).transform(a)
ohe_df= pd.DataFrame(te_array, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
ohe_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15466,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15467,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15468,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,False,False,False,False,False,False
15469,False,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**Discussion**

As I mentioned in the Section 2.2., there are many products (49,688 products total in the Instacart) and many orders (15,471 orders). Therefore, the data is too sparse to catch the meaningful rules to use. To be specific, support value is too low to use in the real world, so it is hard to say the results of the association rule are the meaningful. Instead of using (ordered) products to create rules, I used aisles as the new items by combining merge_df and aisles_df. After finding optimized rules, I will look products.

***
## Section: 3.2 - Create Two-Itemsets
- Create a set of about 50 to 100 two-item sets with highest support. Sort them in decreasing order of support.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [13]:
# If we set min_support to be 0.05, how many orders would need to contain each "frequent" itemset?
np.ceil(0.05 * ohe_df.shape[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


774.0

In [14]:
# create two-itemsets using apriori
from mlxtend.frequent_patterns import apriori

frequent_itemsets_ap = mlx.frequent_patterns.apriori(ohe_df, min_support=0.05, use_colnames=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
frequent_itemsets_ap

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.073557,(baking ingredients)
1,0.163338,(bread)
2,0.066253,(breakfast bakery)
3,0.072329,(butter)
4,0.066835,(candy chocolate)
...,...,...
172,0.051904,"(fresh fruits, eggs, fresh vegetables, package..."
173,0.063732,"(milk, fresh fruits, packaged vegetables fruit..."
174,0.076207,"(packaged cheese, fresh fruits, packaged veget..."
175,0.056945,"(fresh fruits, packaged vegetables fruits, fre..."


In [16]:
# select two-item sets and sort two-item sets with highest support in decreasing
sorted_two_itemsets = frequent_itemsets_ap[frequent_itemsets_ap['itemsets'].apply(lambda x: len(x) == 2)]
sorted_two_itemsets = sorted_two_itemsets.sort_values(by='support', ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
sorted_two_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
82,0.332235,"(fresh fruits, fresh vegetables)"
93,0.289703,"(fresh fruits, packaged vegetables fruits)"
109,0.256545,"(fresh vegetables, packaged vegetables fruits)"
99,0.179950,"(fresh fruits, yogurt)"
91,0.162239,"(packaged cheese, fresh fruits)"
...,...,...
107,0.051257,"(fresh vegetables, other creams cheeses)"
123,0.050999,"(packaged cheese, soy lactosefree)"
117,0.050352,"(packaged cheese, lunch meat)"
92,0.050288,"(fresh fruits, packaged produce)"


**Discussion**

Using a revised dataset - rule based on aisles, I created two-item sets with apriori for better runtime. It is because apriori can prune supersets and save the time. Setting the minimum support is based on my experiments. I set to minimum support 0.05 because there 15,471 orders in the dataset, and 774 orders need to be contained for each frequent item. I think this value looks fine for mining the optimal rules. Also, it can create a set of about 50 to 100 two-item sets. The highest support within two-item sets is about 0.33, and two-item set is {fresh vegetables, fresh fruits}. The lowest support within two-item sets is about 0.05, and two-item set is {fresh vegetables, creal}.

***
# Section: 4 - Generate Rules
- For the two-itemsets created above, create the related rules.
***

In [18]:
# mining some rules based on the confidence
rules_ap = mlx.frequent_patterns.association_rules(frequent_itemsets_ap, metric="confidence", min_threshold=0.7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
rules_ap

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(canned jarred vegetables),(fresh vegetables),0.078405,0.455950,0.059208,0.755153,1.656218,0.023459,2.221996,0.429923
1,(canned meals beans),(fresh fruits),0.072846,0.552647,0.051257,0.703638,1.273214,0.010999,1.509482,0.231446
2,(canned meals beans),(fresh vegetables),0.072846,0.455950,0.053132,0.729370,1.599672,0.019918,2.010310,0.404325
3,(eggs),(fresh fruits),0.150734,0.552647,0.106974,0.709691,1.284168,0.023672,1.540956,0.260561
4,(fresh dips tapenades),(fresh fruits),0.104389,0.552647,0.073428,0.703406,1.272794,0.015738,1.508299,0.239308
...,...,...,...,...,...,...,...,...,...,...
58,"(packaged cheese, fresh vegetables, packaged v...",(fresh fruits),0.090880,0.552647,0.076207,0.838549,1.517332,0.025983,2.770830,0.375031
59,"(fresh fruits, packaged vegetables fruits, soy...",(fresh vegetables),0.072458,0.455950,0.056945,0.785905,1.723666,0.023908,2.541168,0.452639
60,"(fresh vegetables, packaged vegetables fruits,...",(fresh fruits),0.065801,0.552647,0.056945,0.865422,1.565959,0.020581,3.324128,0.386870
61,"(fresh fruits, packaged vegetables fruits, yog...",(fresh vegetables),0.109107,0.455950,0.081766,0.749408,1.643618,0.032018,2.171056,0.439544


***
# Section: 5 - Rule Evaluation
- For the rules created above, find the **Aisle of the Month** and the **set of Products** (that would be given the discount) that would cause the greatest increase in monthly store revenue.
  - This is based on the Business Problem stated at the top of this notebook.
  - Consider:
    - How much will the store's monthly revenue decrease (or increase) due to the change in price for the chosen Products (and its increased sales)?
    - How much will the store's monthly revenue increase (or decrease) due to the increased sales of the associated Products?
***

In [20]:
# sort rules by lift in descending order
two_itemsets = rules_ap[rules_ap['antecedents'].apply(lambda x: len(x) == 1)]
sorted_rules = two_itemsets.sort_values(by='lift', ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
sorted_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
11,(fresh herbs),(fresh vegetables),0.100575,0.455950,0.087195,0.866967,1.901452,0.041338,4.089575,0.527099
0,(canned jarred vegetables),(fresh vegetables),0.078405,0.455950,0.059208,0.755153,1.656218,0.023459,2.221996,0.429923
2,(canned meals beans),(fresh vegetables),0.072846,0.455950,0.053132,0.729370,1.599672,0.019918,2.010310,0.404325
5,(fresh herbs),(fresh fruits),0.100575,0.552647,0.077047,0.766067,1.386178,0.021465,1.912312,0.309744
8,(packaged vegetables fruits),(fresh fruits),0.386400,0.552647,0.289703,0.749749,1.356651,0.076160,1.787618,0.428440
7,(frozen produce),(fresh fruits),0.129403,0.552647,0.094370,0.729271,1.319596,0.022856,1.652400,0.278191
6,(fresh vegetables),(fresh fruits),0.455950,0.552647,0.332235,0.728665,1.318499,0.080255,1.648709,0.444007
10,(yogurt),(fresh fruits),0.247689,0.552647,0.179950,0.726514,1.314607,0.043065,1.635742,0.318109
9,(soy lactosefree),(fresh fruits),0.169866,0.552647,0.121324,0.714231,1.292383,0.027448,1.565438,0.272529
3,(eggs),(fresh fruits),0.150734,0.552647,0.106974,0.709691,1.284168,0.023672,1.540956,0.260561


In [22]:
# extract products from the aisle - fresh herbs
product_names = merged_df2.loc[merged_df2['aisle'] == 'fresh herbs', 'product_name']
print(product_names.unique().tolist())

['Living Organic Cilantro', 'Herb Thyme Clamshell', 'Organic Fresh Basil', 'Baby Dill', 'Parsley, Italian (Flat), New England Grown', 'Mint', 'Living Herbs Potted Living Basil', 'Ginger Root', 'Potted Flat Parsley', 'Organic Mint Bunch', 'Organic Tarragon', 'Basil', 'Organic Poultry Blend', 'Herb Cilantro Clamshell', 'Dill', 'Organic Mixed Herbs', 'Lemongrass Stir In Paste', 'Organic Lemon Thyme', 'Organic Living Basil', 'Herbs Ginger Blend Spices', 'Organic Sage', 'Thyme', 'Organic Mint', 'Potted Thyme', 'Organic Rosemary', 'Cilantro Paste', 'Living Mint', 'Potted Basil', 'Marjoram', 'Sage', 'Lemon Grass', 'Organic Chives', 'Fresh Whole Garlic', 'Dry Arbol Peppers', 'Organic Spearmint', 'Organic Cilantro Bunch', 'Bunched Cilantro', 'Curly Parsley Bunch', 'Organic Cilantro', 'Organic Basil Herb', 'Parsley', 'Lightly Dried Parsley', 'Organic Italian Parsley Bunch', 'Organic Mint Leaves', 'Italian Parsley', 'Fresh Basil', 'Organic Thai Basil', 'Fresh Dill', 'Organic Thyme', 'Organic Dill

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
# the number of sold products in the fresh herbs aisle
len(product_names)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1980

In [24]:
# extract products from the aisle - fresh vegetables (associate products)
product_names = merged_df2.loc[merged_df2['aisle'] == 'fresh vegetables', 'product_name']
print(product_names.unique().tolist())

['European Cucumber', 'Local Living Butter Lettuce', 'Kabocha Squash', 'Garnet Sweet Potato (Yam)', 'Organic Vegetable Tray', 'Fingerling Potato', 'Organic Orange Bell Pepper', 'Green Onion', 'Organic Hass Avocado Bag', 'Sweet Potato (Yam)', 'Yellow Bell Pepper', 'Orange Cauliflower', 'Wild Wonders Medley Tomato', 'Diced Red Onions', 'Organic Green Onions', 'Chayote', 'Red Onions', 'Organic Rainbow Chard Bunch', 'Organic Eggplant', 'Organic Butterhead (Boston, Butter, Bibb) Lettuce', 'Organic Mini Cucumber', 'Chard, Green, New England Grown', 'Organic Brussels Sprouts', 'Organic Broccoli Crowns', 'Rutabaga Organic', 'English Seedless Cucumber', 'Pasilla Pepper', 'Yellow Onions', 'Red Cabbage', 'Boniato Sweet Potato', 'Baby Portabella Mushrooms', 'Broccoli Rabe', 'Mini Peeled Carrots', 'Green Onions', 'Organic Kabocha Squash', 'Premium Super Sweet Corn Bi-Color', 'Organic Red Grape Tomato', 'Dandelion Greens', 'Organic Zucchini Spirals', 'Okra', 'Cauliflower', 'Organic Red Potatoes', 'W

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# the number of sold products in the fresh vegetables aisle
len(product_names)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


18021

In [26]:
# extract products from the aisle - fresh fruits (associate products)
product_names = merged_df2.loc[merged_df2['aisle'] == 'fresh fruits', 'product_name']
print(product_names.unique().tolist())

['Nectarines', 'Cantaloupe', 'Red Seedless Grapes Imported', 'Mini Watermelon', 'Grapes', 'Pink Lady Apple', 'Orange Halo', 'Meyer Lemons', 'Papaya', 'Pineapple', 'Pink Lady Apples', 'Bosc Pear', 'Frozen Organic Blueberries', 'Ambrosia Apples', 'Bag of Oranges', 'Rainier Cherries', 'Organic Granny Smith Apple Bag', 'Bag of Gala Apples', 'Apricot', 'Small Hass Avocado', 'Mexican Papaya', 'Organic Tangellos', 'Pineapple Gold Organic', 'Organic Lemon', 'Royal Gala Apples', 'Berry Valley Blueberries', 'Organic Apples', 'Bag of Lemons', 'Navel Orange', 'Organic Braeburn Apple', 'Gala (3 Lb)', 'Lime', 'Organic Navel Orange', 'Apple Honeycrisp Organic', 'Organic Red Seedless Grapes', 'Organic Goji Berries', 'Fuji  Apples', 'Navel Oranges Bag', 'Granny Smith Apples', 'Large Pink Lady Apples', 'Persimmon Sharon Fruit Cv', 'Driscoll Sunshine Raspberries', 'Artisan Sausage Pineapple Uncured Bacon Hardwood Smoked with Vermont Maple Syrup', 'Black Plum', 'Organic Fuji Apples', 'Dragon Fruit', 'Orga

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# the number of sold products in the fresh fruits aisle
len(product_names)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


17759

**Discussion**

To decide the Aisle of the Month, I consider support, confidence, and lift. Based on the rules from Section4, I choose below two rules:

*   **{fresh herbs} -> {fresh vegetables}**
  * lift is 1.901452, confidence is 0.866967, and support is 0.087195
*   **{fresh herbs} -> {fresh fruits}**
  * lift is 1.386178, confidence is 0.766067, and support is 0.077047

Therefore, **the Aisle of the Month is "*fresh herbs*"**. From orders data, there are 56 types of products. From products data, there are 86 types of products in the Instacart totally.

Set of products in fresh herbs (from orderes data):
> ['Living Organic Cilantro', 'Herb Thyme Clamshell', 'Organic Fresh Basil', 'Baby Dill', 'Parsley, Italian (Flat), New England Grown', 'Mint', 'Living Herbs Potted Living Basil', 'Ginger Root', 'Potted Flat Parsley', 'Organic Mint Bunch', 'Organic Tarragon', 'Basil', 'Organic Poultry Blend', 'Herb Cilantro Clamshell', 'Dill', 'Organic Mixed Herbs', 'Lemongrass Stir In Paste', 'Organic Lemon Thyme', 'Organic Living Basil', 'Herbs Ginger Blend Spices', 'Organic Sage', 'Thyme', 'Organic Mint', 'Potted Thyme', 'Organic Rosemary', 'Cilantro Paste', 'Living Mint', 'Potted Basil', 'Marjoram', 'Sage', 'Lemon Grass', 'Organic Chives', 'Fresh Whole Garlic', 'Dry Arbol Peppers', 'Organic Spearmint', 'Organic Cilantro Bunch', 'Bunched Cilantro', 'Curly Parsley Bunch', 'Organic Cilantro', 'Organic Basil Herb', 'Parsley', 'Lightly Dried Parsley', 'Organic Italian Parsley Bunch', 'Organic Mint Leaves', 'Italian Parsley', 'Fresh Basil', 'Organic Thai Basil', 'Fresh Dill', 'Organic Thyme', 'Organic Dill', 'Flat Parsley, Bunch', 'Organic Poultry Blend Herbs', 'Organic Lemon Thyme Herbs', 'Organic Basil', 'Living Potted Rosemary', 'Organic Parsley']

Store revenue change

* Store revenue decreases due to the change of price for the chosen products. From orders data, products in the fresh herbs aisle sold 1980 times. If all products in fresh herbs aisle discounted 10% and customers buy products on the fresh herbs aisle 5% more frequent, then the store revenue will decrease 5.5% compared to orders data (94.5% of the previous revenue).
* Store revenue increases due to increased sale of associate products. Assume that customers will buy associate products 3% more frequent.
 * Based on the first rule above, customers will buy products from fresh vegetables aisle with a high probability. From orders data, products in the fresh vegetables sold 18,021 times. Therefore, the store revenue will increase 3% compared to orders data.
 * Based on the second rule above, customers will also buy products from fresh fruits aisle with a high probbability. From orders data, products in the fresh fruits sold 17,759 times. Therefore, the store revenue will increase 3% compared to orders data.

Total revenue difference:

> Revenue from orders data: 1980 + 18021 + 17759 = $37,760

> (Expected) Revenue from Aisle of the Month: 1980 x 0.9 x 1.05 + 18,021 x 1.03 + 17,759 x 1.03 = $38,725

Therefore, total revenue difference is ***$965***.





***
# Section: 6 - Conclusions
- Write a paragraph on what you discovered or learned from this homework.
***

**Discussion**

For this homework, I used association rule method to find Aisle of the Month (AOTM). The hardest one in this homework was large dataset to find rules. The number of orders in the orders data is 15,471, and the number of products in the orders data is 20,605. Therefore, the items (products) distribution is too sparse to mining the meaningful rules. Also, the data is high complexity and time consumed to find appropriate rules. Instead of using original orders data, I grouped the orders data with the aisle data. In this case, I used only 134 items to create rules. As a result, I found that the best AOTM is fresh herbs and two rules - {fresh herbs} -> {fresh vegetables} and {fresh herbs} -> {fresh fruits}. There is slight decrease revenue due to discount of prices in fresh herbs aisle, but there is increase revenue due to associate products. So, I can expect $965 increased revenue based on the rules. I can understand how to use association rule in the real world and how association rule is practical for a decision of AOTM and similar problems.

***
# Section: 7 - References
***

* Vernier, S. (2023). chap5-association_analysis[PowerPoint presentation]. WebCampus. https://osu.instructure.com/courses/151965/files/folder/Slides?preview=56082078
* Vernier, S. (2023). GR_Association_Analysis_Example_Code.ipynb [Additional guidance]. WebCampus. https://osu.instructure.com/courses/151965/modules


***
### END-OF-SUBMISSION
***